how this data contribution expands the OBIS holdings

- picking a species from SCCOOS, then pulling all the data for that species from OBIS
- plotting it all together with SCCOOS on a timeslider.

## Load library's

In [ ]:
library(robis)
library(leaflet)
library(tidyverse)

## Get the SCCOOS data from OBIS

In [ ]:
# get SCCOOS occurrence and measurement or fact data
sccoos_dataset_id = "c9aaa0e9-8f6c-4553-a014-a857baba0680"

species = "Lingulodinium polyedra"

sccoos_occur <- occurrence(
    datasetid = sccoos_dataset_id, mof=TRUE
)

## Take a look at response data

In [ ]:
# # make table bigger for review
options(repr.matrix.max.cols=100)
head(sccoos_occur,n = 1)

## Note measurement or fact data are in `mof` column.

In [ ]:
# here are the mof observations associated with the first occurrence record above.
head(sccoos_occur$mof,n = 1)
# note measurementValue, measurementUnit and measurementType columns.

## Create a map of all SCCOOS observations for the dataset

In [ ]:
# create a map
library(htmlwidgets)
library(IRdisplay)

m = map_leaflet(sccoos_occur)
m
saveWidget(m, "sccoos_data.html", selfcontained = FALSE)
display_html("<iframe src=\"sccoos_data.html\"></iframe>")

## Query OBIS for _Lingulodinium polyedra_ and map out all observations (excluding SCCOOS' contributions)
Now that we have all the SCCOOS occurrence and mof observations, we'd like to look at one species and see SCCOOS' contributions to OBIS for that species.

In [ ]:
# get all Lingulodinium polyedra occurrences from OBIS
lingpoly_occur <- occurrence(
    scientificname = species
)

# or by taxon id from AphiaID: urn:lsid:algaebase.org:taxname:233592
# could use rtaxize 'get_wormsid' here or obistools 'match_taxa' to search on taxonid from species name? 
# lingpoly_occur <- occurrence(
#   taxonid = '233592'
# )

# remove sccoos entries from full occurrence response
lingpoly_occur <- lingpoly_occur[!lingpoly_occur$dataset_id==sccoos_dataset_id,]

# create a map of all lingulodinium polyedra, excluding sccoos contributions
m = map_leaflet(lingpoly_occur)
m
saveWidget(m, "lingpoly_data.html", selfcontained = FALSE)
display_html("<iframe src=\"lingpoly_data.html\"></iframe>")

## Subset SCCOOS response for species and move the mof data into occurrence data table

In [ ]:
# build a character list of all the unique measurement or fact types.
sccoos_lingpoly_occur <- sccoos_occur[ which(sccoos_occur$scientificName == species), ]

for (j in 1:length(sccoos_lingpoly_occur$mof)){
    if (j == 1){
        measurementTypes <- sccoos_lingpoly_occur$mof[[j]]$measurementType
    } else {
    measurementTypes <- unique(c(measurementTypes, sccoos_lingpoly_occur$mof[[j]]$measurementType))
        }
}

# add columns for all potential mof types
sccoos_lingpoly_occur[,measurementTypes] <- ""

## Populate mof_tbl with mof observations
for (i in 1:length(sccoos_lingpoly_occur$mof)){
    for (j in 1:length(sccoos_lingpoly_occur$mof[[i]])){
        for (var in measurementTypes){
            if ( length(sccoos_lingpoly_occur$mof[[i]]$measurementType[j]) == 0 || is.na(sccoos_lingpoly_occur$mof[[i]]$measurementType[j])) {
            } else{
                if ( sccoos_lingpoly_occur$mof[[i]]$measurementType[j] == var ){
                    #print(paste("sccoos_lingpoly_occur$mof[[",i,"]]$measurementType[",j,"] =", sccoos_lingpoly_occur$mof[[i]]$measurementType[j]," = ", sccoos_lingpoly_occur$mof[[i]]$measurementValue[j]))
                    sccoos_lingpoly_occur[i,var] <- sccoos_lingpoly_occur$mof[[i]]$measurementValue[j]
                } else {
                    sccoos_lingpoly_occur[i,var] <- NA
                }     
            }
        }
    }
}

# convert mofs to doubles
measurementTypes <- c("organismQuantity", "sss", "sst", "bathymetry", measurementTypes)
sccoos_lingpoly_occur[ ,measurementTypes] <- apply(sccoos_lingpoly_occur[ ,measurementTypes], 2, function(x) as.double(x))
sccoos_lingpoly_occur$dateTime <- as.Date(sccoos_lingpoly_occur$eventDate, format = "%Y-%m-%dT%H:%M:%SZ", tz=utc)

# collect column names and drop 'mof' column from list (we don't need it anymore):
cols <- colnames(sccoos_lingpoly_occur)
cols <- cols[cols!='mof']
sccoos_lingpoly_occur <- sccoos_lingpoly_occur[cols]

# What data are in our new data table?
glimpse(sccoos_lingpoly_occur)

## Group observations by location and print out some statistics

In [ ]:
# create a grouped data table
by_location <- sccoos_lingpoly_occur %>% group_by(locationID)

by_location %>%
    summarise(
        n = n(),
        Temp = mean(Temp, na.rm = TRUE),
        Salinity = mean(Salinity, na.rm = TRUE),
        organismQuantity = max(organismQuantity, na.rm = TRUE)
    )



## Create time-series plots for observations, colored by location

In [ ]:
library(ggplot2)

p1 <- ggplot(by_location, aes(x=dateTime, y=Temp, col=locationID)) +
geom_line()

p2 <- ggplot(by_location, aes(x=dateTime, y=organismQuantity, col=locationID)) +
geom_line()

p1
p2

## Make a map of all occurrences with SCCOOS obs as cluster pins

In [ ]:
m <- map_leaflet(lingpoly_occur) %>% 
  addTiles() %>% 
  addMarkers(
    sccoos_lingpoly_occur$decimalLongitude, sccoos_lingpoly_occur$decimalLatitude,
    clusterOptions = markerClusterOptions(),
  )
m
saveWidget(m, "lingpoly_data_sub.html", selfcontained = FALSE)
display_html("<iframe src=\"lingpoly_data_sub.html\"></iframe>")

## Make a master map.
This map has all the _Lingulodinium polyedra_ obeservations from OBIS with clustered points for the SCCOOS observations. Each cluster opens to pins for each mof observed at that location. Each specfic pin opens a time-series plot of the mof observation.

In [ ]:
m <- map_leaflet(lingpoly_occur)
# iterate through each site
for (site in unique(sccoos_lingpoly_occur$locationID)) {
    
    # filter for the site
    site_filt <- filter(sccoos_lingpoly_occur, locationID == site)
    
    # create time-series plot for each measurement type
    pop <- character()
    #color <- 'blue'
    for (variable in measurementTypes){
        if ( !all(is.na(site_filt[,variable])) ) {
            ggplot(site_filt, aes_string(x="dateTime", y=variable)) + geom_point() + ggtitle(site)
            image_name <- paste0(site,variable,"_ts.png")
            ggsave(image_name, width = 2, height = 2)
            pop1 <- paste0("<img src=\"",image_name,"\" />")
            pop <- c(pop,pop1)
            #c1 <- 'red'
            #color <- c(color,c1)
            }
    }

    m <- addTiles(m) %>%
         addMarkers(
            unique(site_filt$decimalLongitude), unique(site_filt$decimalLatitude),
            clusterOptions=markerClusterOptions(),
            popup = pop
        )

    }
filename <- paste0("map.html")
saveWidget(m, filename, selfcontained = FALSE)
html_file = paste0("<iframe src=\"",filename,"\"></iframe>")
display_html(html_file)